## Relational Database Schema using SQLite 
Authors -

    Srijith Srinath
    Vinu Prasad Bhambore

In [1]:
#Importing packages

import sqlite3
import csv
import pandas as pd

In [2]:
#Establishing connection to the database
connection = sqlite3.connect('Shooting.db') #Creating the database (Or it will connect if it already exists)
cursor = connection.cursor() #Creating the cursor to access the database

In [3]:
df = pd.read_csv("zomato_clean_openrefine.csv", encoding = "ISO-8859-1") #Using pandas to read csv file into a dataframe
print("The shape of the dataframe is:  ", df.shape)

The shape of the dataframe is:   (12467, 13)


In [4]:
#Stripping whitespace from headers and converting title case to lower case
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [5]:
#### NOTE: Only run this block for thr first time while running this program. 
####    Running it multiple times will throw an error saying the table already exists

#Adding data into database
#df.to_sql("Shooting_table", connection) #Creating the table using the dataframe

In [6]:
#Listing all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('Shooting_table',)]


In [7]:
#Looking at the schema of the table
cursor.execute("PRAGMA table_info(Shooting_table);")
schema = cursor.fetchall()
for items in schema:
    print(items)

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'name', 'TEXT', 0, None, 0)
(2, 'address', 'TEXT', 0, None, 0)
(3, 'online_order', 'TEXT', 0, None, 0)
(4, 'book_table', 'TEXT', 0, None, 0)
(5, 'rate', 'TEXT', 0, None, 0)
(6, 'votes', 'INTEGER', 0, None, 0)
(7, 'phone', 'TEXT', 0, None, 0)
(8, 'location', 'TEXT', 0, None, 0)
(9, 'rest_type', 'TEXT', 0, None, 0)
(10, 'dish_liked', 'TEXT', 0, None, 0)
(11, 'cuisines', 'TEXT', 0, None, 0)
(12, 'approx_cost(for two people)', 'INTEGER', 0, None, 0)
(13, 'reviews_list', 'TEXT', 0, None, 0)


### Integrity Constraints

1. There is only one table in the sqlite database, which means that there are no foreign key relationships. There will be no integrity constraints that check for these relationships.

In [8]:
#Looking at the schema of the table
cursor.execute("SELECT COUNT (*) FROM Shooting_table;")
rows = cursor.fetchall()
cursor.execute("SELECT COUNT (*) FROM pragma_table_info('Shooting_table');")
columns = cursor.fetchall()
print("Rows:    ", rows[0][0])
print("Columns: ", columns[0][0], "(including index column)")
print("---------------------------------------------------")

Rows:     12467
Columns:  14 (including index column)
---------------------------------------------------


2. Check uniqueness of the composite key: name,address

In [9]:
cursor.execute("SELECT name, address, rate, COUNT(*) AS x FROM Shooting_table GROUP BY name, address, rate HAVING x > 1")
distinct_ids = cursor.fetchall()
print("Number of ids: ",len(distinct_ids),"\n")
print(distinct_ids)
print("---------------------------------------------------")

Number of ids:  4 

[("Am's Cafe Recreate", 'Shop 2, Vp Road, Opposite To Sandhya Theatre, Madiwala, Btm, Bangalore', '3.3', 2), ('Cupcake Bliss', '199, Second Floor, C Cross, Koramangala 7th Block, Bangalore', 'MISSING', 2), ('Delish Chinese Fast Food', '4th Cross, Cmr Law Collage, Ombr Layout, Banaswadi, Bangalore', '3.2', 2), ('Good Meals', '#325, Ground Floor, 6th Main Road, Hal 2nd Stage, Indiranagar, Bangalore', 'MISSING', 2)]
---------------------------------------------------


3. Checking if the ratings are between 0 and 5

In [10]:
cursor.execute("SELECT COUNT(*) FROM Shooting_table WHERE rate < '0' and rate > '5';")
ratings = cursor.fetchall()
print("Invalid ratings count: ", ratings[0][0])

Invalid ratings count:  0


4. Checking if number of votes for any restaurant are negative

In [11]:
cursor.execute("SELECT COUNT(*) FROM Shooting_table WHERE votes < 0;")
negative_votes = cursor.fetchall()
print("Resturants with negative votes : ", negative_votes[0][0])

Resturants with negative votes :  0


5. Checking of the approx amount for two is negative for some restaurant

In [12]:
cursor.execute("SELECT COUNT(*) FROM Shooting_table WHERE \"approx_cost(for two people)\" < 0 and \"approx_cost(for two people)\" != -99;")
zero_cost = cursor.fetchall()
print("Restuarants with negative amount : ", zero_cost[0][0])

Restuarants with negative amount :  0


In [13]:
connection.close() #Closing the connection to the database